In [ ]:
%pip install azureml-sdk[databricks]
%pip install azureml-mlflow
%pip install rpy2

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.azureml
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice
from random import randint
from azureml.core.environment import Environment,CondaDependencies
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice.aks import AksServiceDeploymentConfiguration
from azureml.core.resource_configuration import ResourceConfiguration

interactive_auth = InteractiveLoginAuthentication(tenant_id="xxxxx")
subscription_id = "xxxxx" #you should be owner or contributor
resource_group = "xxxxx" #Resource group name. NOTE: you should be owner or contributor
workspace_name = "xxxxx" #AzureML workspace name
aks_compute_name = "xxxxx"
experiment_name = "xxxxx" # Cab be any name and will be displayed in the Azure ML UI
workspace_region = 'xxxxx'

workspace = Workspace.get(name = workspace_name,
                          location = workspace_region,
                          resource_group = resource_group,
                          subscription_id = subscription_id,
                          auth=interactive_auth)

mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)
aks_target = AksCompute(workspace,aks_compute_name)

In [ ]:
# Sample R function to use be used as part of scoring function
# -----------------------------------------------------------
# Replace this function with your R pre-processing steps
# -----------------------------------------------------------

r_init_code = """fahrenheit_to_celsius <- function(temp_F) {
  temp_C <- (temp_F - 32) * 5 / 9
  return(temp_C)
}"""

with open('score.R','w') as f:
  f.write(r_init_code)

In [ ]:
# Specify customer Dockerfile to build environment with R. Note R package needs to be compiled with --enable-R-shlib option
dockerfile = r"""FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
RUN mkdir /usr/share/man/man1 && apt-get update && apt-get install --no-install-recommends -y openjdk-8-jdk
ENV JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64/
ENV DEBIAN_FRONTEND=noninteractive
RUN apt-get update \
  && apt-get install --yes \
    libssl-dev \
    libfuse-dev \
    python3 python3-pip \
    wget \
    openjdk-8-jdk \
    build-essential gfortran libreadline-dev libxml2-dev libcurl4-openssl-dev libpcre2-dev libbz2-dev liblzma-dev \
  && apt-get clean \
  && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/*
# Conda Environment
ENV MINICONDA_VERSION py37_4.9.2
ENV PATH /opt/miniconda/bin:$PATH
RUN wget -qO /tmp/miniconda.sh https://repo.continuum.io/miniconda/Miniconda3-${MINICONDA_VERSION}-Linux-x86_64.sh && \
    bash /tmp/miniconda.sh -bf -p /opt/miniconda && \
    conda clean -ay && \
    rm -rf /opt/miniconda/pkgs && \
    rm /tmp/miniconda.sh && \
    find / -type d -name __pycache__ | xargs rm -rf
RUN wget -qO R-4.0.4.tar.gz https://cran.r-project.org/src/base/R-4/R-4.0.4.tar.gz && \
    tar -xvf R-4.0.4.tar.gz && \
    cd R-4.0.4 && \
    ./configure --enable-R-shlib --with-x=no --without-recommended-packages && \
    make -j4 && make install && rm -rf /R-4.0.4.tar.gz
ENV LD_LIBRARY_PATH="/usr/local/lib/R/lib:$LD_LIBRARY_PATH"
RUN ldconfig
RUN R -e "install.packages(c('dplyr'), repos = 'https://cloud.r-project.org/')"
RUN R -e "install.packages(c('conflicted'), repos = 'https://cloud.r-project.org/')"
"""
with open('Dockerfile', 'w') as f:
  f.write(dockerfile)

In [ ]:
# Python entry script to translate pandas df to R df and vice-versa
# -----------------------------------------------------------
# Replace run function with your python pre-processing steps. e.g. parse input json etc
# Note: both R pre-process script and R rds file are loaded as models
# -----------------------------------------------------------
score_py = """import readline
import joblib
import os
import json
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from azureml.core.model import Model

# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global r_model_path,r_entry_script, score_function_r

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    r_model_path = Model.get_model_path(model_name='my-r-model')    
    r_entry_script = Model.get_model_path(model_name='my-r-script')
    
    # Defining the R script and loading the instance in Python
    r = ro.r
    r['source'](r_entry_script)
    # Loading the function we have defined in R.
    score_function_r = ro.globalenv['fahrenheit_to_celsius']

# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and validates the incoming payload against
# the example input you provide here. This will also generate a Swagger
# API document for your web service.
@input_schema('data', NumpyParameterType(np.array([100])))
@output_schema(NumpyParameterType(np.array([1])))
def run(data):
    # sample data
    pd_df = pd.DataFrame(np.array([100]))
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_from_pd_df = ro.conversion.py2rpy(pd_df)

    # r_from_pd_df
    df_result_r = score_function_r(r_from_pd_df)

    with localconverter(ro.default_converter + pandas2ri.converter):
        pd_from_r_df = ro.conversion.rpy2py(df_result_r)

    return pd_from_r_df.to_json()"""

with open('score.py','w') as f:
    f.write(score_py)

In [ ]:
%r
# Generating random data (3000) to be loaded as part of the scoring
# -----------------------------------------------------------
# Create this class.rds file from your existing R model
# -----------------------------------------------------------
RandomNum <- runif(3000, 1, 5000)
save(RandomNum,file="/databricks/driver/class.rds")

In [ ]:
# Testing of python rpy2 package
# -----------------------------------------------------------
# Test locally to check the R pre-process script works as expected
# -----------------------------------------------------------
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import json
from rpy2.robjects.conversion import localconverter

from azureml.core.model import Model
r = ro.r
r['source']("score.R")
# Loading the function we have defined in R.
score_function_r = ro.globalenv['fahrenheit_to_celsius']

# Create pandas dataframe
pd_df = pd.DataFrame(np.array([100]))

# Create R datafram from pandas dataframe
with localconverter(ro.default_converter + pandas2ri.converter):
  r_from_pd_df = ro.conversion.py2rpy(pd_df)

# pass R dataframe to R scoring function 
df_result_r = score_function_r(r_from_pd_df)

# Create pandas dataframe from R dataframe
with localconverter(ro.default_converter + pandas2ri.converter):
  pd_from_r_df = ro.conversion.rpy2py(df_result_r)

pd_from_r_df.to_json()

In [ ]:
# Create, register and build a custom R image from custome Docker image
myrenv_py = Environment(name="custom_env_r")
myrenv_py.inferencing_stack_version='latest'
myrenv_py.docker.enabled = True
myrenv_py.docker.base_image = None
myrenv_py.docker.base_dockerfile = "./Dockerfile"
myrenv_py.register(workspace=workspace)

conda_dep_pkgs=['joblib','pandas','tzlocal']
pip_pkgs=['azureml-defaults', 'inference-schema','rpy2==3.4.5']
conda_dep = CondaDependencies()

# Install additional conda packages as required
for conda_dep_pkg in conda_dep_pkgs:
  conda_dep.add_conda_package(conda_package=conda_dep_pkg)

# Install additional pip packages as required
for pip_pkg in pip_pkgs:
  conda_dep.add_pip_package(pip_package=pip_pkg)

myrenv_py.python.conda_dependencies=conda_dep

In [ ]:
# Create webservice and deploy the model to AKS
prod_webservice_name = "r-model-prod-01"
prod_webservice_deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
inference_config = InferenceConfig(entry_script='score.py',environment=myrenv_py)

azureml_r_model = Model.register(workspace=workspace,
                       model_name='my-r-model',                # Name of the registered model in your workspace.
                       model_path='./class.rds',  # Local file to upload and register as a model.
                       description='R Model RDS',
                       tags={'area': 'azureml', 'type': 'databricks notebook', 'language': 'R'})

azureml_r_entry_script = Model.register(workspace=workspace,
                       model_name='my-r-script',                # Name of the registered model in your workspace.
                       model_path='./score.R',  # Local file to upload and register as a model.
                       description='R model scoring scrpt',
                       tags={'area': 'azureml', 'type': 'databricks notebook', 'language': 'R'})

service = Model.deploy(workspace=workspace,
                      name=prod_webservice_name,
                      models=[azureml_r_model,azureml_r_entry_script],
                      inference_config=inference_config,
                      deployment_config=prod_webservice_deployment_config,
                      deployment_target = aks_target,
                      overwrite=True)

In [ ]:
# Check the status of the deployment, note first deployment can take upto 20 min, since it includes building a custom docker image
print(service.get_logs())

In [ ]:
import json
import requests

scoring_uri = service.scoring_uri
key, _ = service.get_keys()
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"
data = {"data":[100]}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)